# US Twitter Sentiment on Climate Change



###1. Import Libraries

In [0]:
!pip install -q langdetect

import pandas as pd
from sklearn.model_selection import train_test_split

import numpy as np
import re
import scipy as sp
import time

from langdetect import detect_langs
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from wordcloud import WordCloud, STOPWORDS
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
import re

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold

###2. Read Dataset to DataFrame

In [0]:
# Read Train and Test Dataset
df_tweet = pd.read_csv('kaggleTweets_cleaned_all_dupl_withSW.csv')

Drive already mounted at /G; to attempt to forcibly remount, call drive.mount("/G", force_remount=True).


###3. Data Exploratory Analysis 

In [0]:
print (df_tweet.info())
df_tweet.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40219 entries, 0 to 40218
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            40219 non-null  int64 
 1   sentiment        40219 non-null  int64 
 2   message          40219 non-null  object
 3   tweetid          40219 non-null  int64 
 4   message_toc      40219 non-null  object
 5   message_lem      40219 non-null  object
 6   message_len_str  40219 non-null  object
 7   language         40219 non-null  object
dtypes: int64(3), object(5)
memory usage: 2.5+ MB
None


,index,sentiment,message,tweetid,message_toc,message_lem,message_len_str,language
0,0,-1,tiniebeany climate change is an interesting h...,792927353886371840,"['tiniebeany', 'climate', 'change', 'is', 'an'...","['tiniebeany', 'climate', 'change', 'is', 'an'...","['tiniebeany', 'climate', 'change', 'is', 'an'...",en
1,1,1,rt natgeochannel watch beforetheflood right he...,793124211518832641,"['rt', 'natgeochannel', 'watch', 'beforetheflo...","['rt', 'natgeochannel', 'watch', 'beforetheflo...","['rt', 'natgeochannel', 'watch', 'beforetheflo...",en
2,2,1,fabulous dicaprio dicaprio s film on climate c...,793124402388832256,"['fabulous', 'dicaprio', 'dicaprio', 's', 'fil...","['fabulous', 'dicaprio', 'dicaprio', 's', 'fil...","['fabulous', 'dicaprio', 'dicaprio', 's', 'fil...",en
3,3,1,rt mick fanning just watched this amazing docu...,793124635873275904,"['rt', 'mick', 'fanning', 'just', 'watched', '...","['rt', 'mick', 'fanning', 'just', 'watched', '...","['rt', 'mick', 'fanning', 'just', 'watched', '...",en
4,4,2,rt cnalive pranita biswasi a lutheran from odi...,793125156185137153,"['rt', 'cnalive', 'pranita', 'biswasi', 'a', '...","['rt', 'cnalive', 'pranita', 'biswasi', 'a', '...","['rt', 'cnalive', 'pranita', 'biswasi', 'a', '...",en


###4. Train_Validation_Test Data Split

In [0]:
df_tweet_copy = df_tweet.copy()

def data_split(df):
    df_train, df_test = train_test_split(df, test_size=0.2, random_state=5678) 
    return df_train, df_test

df_train, df_test= data_split(df_tweet_copy)
print (df_train.info())
print (df_test.info())
print (type(df_train))


<class 'pandas.core.frame.DataFrame'>
Int64Index: 32175 entries, 21241 to 34932
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            32175 non-null  int64 
 1   sentiment        32175 non-null  int64 
 2   message          32175 non-null  object
 3   tweetid          32175 non-null  int64 
 4   message_toc      32175 non-null  object
 5   message_lem      32175 non-null  object
 6   message_len_str  32175 non-null  object
 7   language         32175 non-null  object
dtypes: int64(3), object(5)
memory usage: 2.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8044 entries, 16310 to 17784
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            8044 non-null   int64 
 1   sentiment        8044 non-null   int64 
 2   message          8044 non-null   object
 3   tweetid          8044 non-null   int64 
 4

In [0]:
train_x = df_train.message_len_str.to_list()
train_y = df_train.sentiment.to_list()
test_x = df_test.message_len_str.to_list()
test_y = df_test.sentiment.to_list()
print (len(train_x))
print (len(train_y))
print (len(test_x))
print (len(test_y))
print (type(train_x))

32175
32175
8044
8044
<class 'list'>


###5. Model Train, Validate and Hyperparameter Tuning

####5.1 MultinomialNB model 

In [0]:
from sklearn.naive_bayes import MultinomialNB
skf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 999)

def MultinomialNB_model(features, target):

    pipe = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('cls', MultinomialNB()),
    ])
    
    # gridsearch to tune hyperparameters 
    p_grid = {
    'vect__ngram_range': [(1, 1),(1,2)],
    'tfidf__use_idf': (True, False),
    'cls__alpha': [0.001,0.005,0.01,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'cls__class_prior' : [None, [.1,.9],[.2, .8]], 
    'cls__fit_prior' : [True, False]
    }

    classifier = GridSearchCV(estimator=pipe, param_grid=p_grid, verbose=0, scoring = 'accuracy', n_jobs=-1, cv=skf.split(features, target)).fit(features, target)
    return classifier.best_estimator_, classifier.best_score_, classifier.best_params_, classifier.cv_results_

 
%time NB_best_estimator, NB_best_score,  NB_best_params, NB_cv_results = MultinomialNB_model(train_x, train_y)
print("The training accuracy score are: {:.2f}".format(NB_best_score))
print("The best params are: -")
for param_name in sorted(NB_best_params.keys()):
  print("%s: %r" % (param_name, NB_best_params[param_name]))

CPU times: user 7min 59s, sys: 5.03 s, total: 8min 4s
Wall time: 30min 33s
The training accuracy score are: 0.72
The best params are: -
cls__alpha: 0.1
cls__class_prior: None
cls__fit_prior: False
tfidf__use_idf: True
vect__ngram_range: (1, 2)


In [0]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

pred_y = NB_best_estimator.predict(test_x)

print('accuracy %s' % accuracy_score(pred_y, test_y))
print(classification_report(test_y, pred_y))

accuracy 0.7166832421680756
              precision    recall  f1-score   support

          -1       0.72      0.48      0.58       762
           0       0.59      0.38      0.46      1375
           1       0.74      0.84      0.79      4154
           2       0.71      0.78      0.74      1753

    accuracy                           0.72      8044
   macro avg       0.69      0.62      0.64      8044
weighted avg       0.71      0.72      0.70      8044



####5.2 Logit Regression Model

In [0]:
from sklearn.linear_model import LogisticRegression
skf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 999)

def LogitReg_model(features, target):
    #features = df.message.to_list()
    #target = df.sentiment.to_list()

    pipe = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('cls', LogisticRegression()),
    ])
    
    # gridsearch to tune hyperparameters 
    p_grid = {
    'vect__ngram_range': [(1, 1), (1,2)],
    'vect__max_features': [2000000],
    'vect__vocabulary':[None],
    'tfidf__use_idf': (True, False),
    'cls__penalty' : ['l1', 'l2'],
    'cls__C' : [0.01, 0.1, 1, 10, 100, 1000],
    'cls__solver' : ['newton-cg', 'lbfgs','liblinear']
    }

    classifier = GridSearchCV(estimator=pipe, param_grid=p_grid, verbose=0, scoring = 'accuracy', n_jobs=-1, cv=skf.split(features, target)).fit(features, target)
    return classifier.best_estimator_, classifier.best_score_, classifier.best_params_,classifier.cv_results_

%time LG_best_estimator, LG_best_score,  LG_best_params, LG_cv_results = LogitReg_model(train_x, train_y)
print("The training accuracy score are: {:.2f}".format(LG_best_score))
print("The best params are: -")
for param_name in sorted(LG_best_params.keys()):
  print("%s: %r" % (param_name, LG_best_params[param_name]))

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


CPU times: user 4min 25s, sys: 26.6 s, total: 4min 51s
Wall time: 1h 1min 12s
The training accuracy score are: 0.73
The best params are: -
cls__C: 100
cls__penalty: 'l2'
cls__solver: 'newton-cg'
tfidf__use_idf: True
vect__max_features: 2000000
vect__ngram_range: (1, 2)
vect__vocabulary: None


In [0]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

pred_y = LG_best_estimator.predict(test_x)

print('accuracy %s' % accuracy_score(pred_y, test_y))
print(classification_report(test_y, pred_y))
#print (NB_best_estimator.get_params)

accuracy 0.7432869219293884
              precision    recall  f1-score   support

          -1       0.73      0.52      0.61       762
           0       0.62      0.45      0.52      1375
           1       0.76      0.87      0.81      4154
           2       0.77      0.78      0.78      1753

    accuracy                           0.74      8044
   macro avg       0.72      0.65      0.68      8044
weighted avg       0.74      0.74      0.73      8044



###5.3 Linear Support Vector Machine Model




In [0]:
from sklearn.linear_model import SGDClassifier
skf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 999)

def SGD_model(features, target):

    pipe = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('cls', SGDClassifier(penalty='l2', random_state=8, tol=None)),
    ])
    
    # gridsearch to tune hyperparameters 
    p_grid = {
    'vect__ngram_range': [(1,2)], #[(1, 1),(1,2)],
    'tfidf__use_idf': (True, False),
    'cls__alpha' : [1e-5, 1e-6],    #[1e-1,1e-2,1e-3,1e-4,1e-5],
    'cls__max_iter' : [2000,3000],  #[1,10,100,1000,2000],
    'cls__loss' : ['hinge','modified_huber']  
    }

    classifier = GridSearchCV(estimator=pipe, param_grid=p_grid, verbose=0, scoring = 'accuracy', n_jobs=-1, cv=skf.split(features, target)).fit(features, target)
    return classifier.best_estimator_, classifier.best_score_, classifier.best_params_, classifier.cv_results_

%time SGD_best_estimator, SGD_best_score,  SGD_best_params, SGD_cv_results = SGD_model(train_x, train_y)
print("The training accuracy score are: {:.2f}".format(SGD_best_score))
print("The best params are: -")
for param_name in sorted(SGD_best_params.keys()):
  print("%s: %r" % (param_name, SGD_best_params[param_name]))


CPU times: user 2min 17s, sys: 773 ms, total: 2min 17s
Wall time: 1h 13min 53s
The training accuracy score are: 0.73
The best params are: -
cls__alpha: 1e-05
cls__loss: 'modified_huber'
cls__max_iter: 3000
tfidf__use_idf: True
vect__ngram_range: (1, 2)


In [0]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

sgd_pred_y = SGD_best_estimator.predict(test_x)

print('accuracy %s' % accuracy_score(sgd_pred_y, test_y))
print(classification_report(test_y, sgd_pred_y))

accuracy 0.7420437593237196
              precision    recall  f1-score   support

          -1       0.73      0.54      0.62       762
           0       0.62      0.44      0.51      1375
           1       0.76      0.87      0.81      4154
           2       0.76      0.78      0.77      1753

    accuracy                           0.74      8044
   macro avg       0.72      0.65      0.68      8044
weighted avg       0.73      0.74      0.73      8044

